In [1]:
from WirelessSystem.system import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import keras

Using TensorFlow backend.


In [4]:
import os
import numpy as np
from SigGAN.train import Trainer

In [5]:
import tensorflow as tf

In [6]:
def norm(sig_u):
    pwr = np.sqrt(np.mean(np.sum(sig_u**2,axis = -1),axis = -1))
    sig_u = sig_u/pwr[:,None,None]
    print(sig_u.shape)
    return sig_u

In [7]:
def shuffle(vec1,vec2,seed = 0):
    np.random.seed(0)
    shfl_indx = np.arange(vec1.shape[0])
    np.random.shuffle(shfl_indx)
    shfl_indx = shfl_indx.astype('int')
    vec1 = vec1[shfl_indx]
    vec2 = np.copy(vec2[shfl_indx])
    return vec1,vec2

In [8]:
def split(vec,n_train,n_valid,n_test):
    vec_train = vec[0:n_train]
    vec_valid = vec[n_train:n_train+n_valid]
    vec_test = vec[n_train+n_valid:]
    return vec_train,vec_valid,vec_test

In [9]:
n_symbols = 256 // 2
n_authorized = 5
n_unauthorized = 5

In [10]:
rf_system = RFSystem(n_authorized = n_authorized, n_unauthorized = n_unauthorized)

In [11]:
sig_auth_, txid_auth = rf_system.get_n_received_symbol_blocks(1000, n_symbols, authorized = 0)
sig_unauth_, txid_unauth = rf_system.get_n_received_symbol_blocks(1000, n_symbols, authorized = 1)
sig_impersonate_, _ = rf_system.get_n_received_symbol_blocks(10, n_symbols, authorized = 2)

In [12]:
sig_auth=sig_auth_
sig_unauth=sig_unauth_

In [13]:
sig_impersonate_ad_, _ = rf_system.get_n_received_symbol_blocks(1000, n_symbols, authorized = 2)
sig_impersonate_ad=sig_impersonate_ad_

In [14]:
sig_impersonate=sig_impersonate_.reshape((-1,2))

In [15]:
sig_impersonate.shape

(2560, 2)

In [16]:
np.shape(sig_auth)

(1000, 256, 2)

In [66]:
# hyper parameters
B = 64 # batch size
T = 256 # Max length of sentence
g_H = 100 # Generator LSTM hidden size
g_lr = 1e-3
d_dropout = 0.5 # dropout ratio
d_lr = 1e-3

n_sample=2 # Number of Monte Calro Search
generate_samples = 2000 # Number of generated sentences

# Pretraining parameters
g_pre_lr = 1e-3
d_pre_lr = 1e-3
g_pre_epochs= 60
d_pre_epochs = 1

In [109]:
trainer = Trainer(B, T, n_authorized, g_H, d_dropout, g_lr=g_lr, d_lr=d_lr, n_sample=n_sample, generate_samples=generate_samples)

In [110]:
sig_rd = np.concatenate([sig_auth,sig_unauth])
txid_rd = np.concatenate([txid_auth,np.ones_like(txid_unauth)*n_authorized])

In [111]:
sig_rd, txid_rd = shuffle(sig_rd, txid_rd)

In [112]:
txid_disc = txid_rd == n_authorized
txid_disc = np.invert(txid_disc)

In [113]:
print(np.unique(txid_rd))

[0 1 2 3 4 5]


In [114]:
test_frac = 0.1
valid_frac  = 0.2

n_samples  = sig_rd.shape[0]

n_test = int(test_frac*n_samples)
n_valid = int(valid_frac*n_samples)
n_train = n_samples - n_test - n_valid

In [115]:
sig_rd_train,sig_rd_valid,sig_rd_test=split(sig_rd,n_train,n_valid,n_test)

In [116]:
out_train,out_valid,out_test=split(txid_disc,n_train,n_valid,n_test)

In [117]:
trainer.pre_train_discriminator(d_epochs=30, d_pre_sig_train=sig_rd_train, d_pre_out_train=out_train, d_pre_sig_valid=sig_rd_valid, d_pre_out_valid=out_valid, lr=1e-3)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 2)       0                                            
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 256, 2, 1)    0           input_3[0][0]                    
__________________________________________________________________________________________________
1_conv1 (Conv2D)                (None, 256, 2, 16)   32          reshape_3[0][0]                  
__________________________________________________________________________________________________
1_conv2 (Conv2D)                (None, 256, 2, 16)   1552        1_conv1[0][0]                    
__________________________________________________________________________________________________
1_bn (Batc

In [118]:
print(trainer.test_discriminator(sig_rd_test, out_test))

200/200 [==============================] - 0s 93us/step
0.9453220701217652


In [119]:
print(trainer.test_discriminator(sig_impersonate_ad, np.zeros((sig_impersonate_ad.shape[0],))))

1000/1000 [==============================] - 0s 87us/step
0.8012445406913757


In [120]:
np.mean(trainer.predict_discriminator(sig_auth))

0.4617978

In [121]:
np.mean(trainer.predict_discriminator(sig_unauth))

0.42659783

In [122]:
np.mean(trainer.predict_discriminator(sig_impersonate_ad))

0.44439816

In [123]:
sig_impersonate.shape

(2560, 2)

In [124]:
n_samples_im  = sig_impersonate.shape[0]

n_test_im = int(test_frac*n_samples_im)
n_valid_im = int(valid_frac*n_samples_im)
n_train_im = n_samples_im - n_test_im - n_valid_im

In [125]:
sig_im_train,sig_im_valid,sig_im_test=split(sig_impersonate,n_train_im,n_valid_im,n_test_im)

In [126]:
sig_im_train_re = sig_im_train.reshape((-1, 1, 2))
sig_im_train_re_t = sig_im_train.reshape((-1, 2))

In [127]:
sig_im_valid_re = sig_im_valid.reshape((-1, 1, 2))
sig_im_valid_re_t = sig_im_valid.reshape((-1, 2))

In [128]:
sig_im_test_re = sig_im_test.reshape((-1, 1,  2))

In [129]:
# def GeneratorPretraining():
#     def sampling(args):
#         mean_mu, log_var = args
#         epsilon = K.random_normal(shape=K.shape(mean_mu), mean=0., stddev=1.) 
#         return mean_mu + K.exp(log_var/2)*epsilon
#     def sampling_alt(args):
#         mean_mu, log_var = args
#         normal_dist = tf.distributions.Normal(mean_mu, tf.exp(log_var/2))
#         return normal_dist.sample()
    
#     input=Input(shape=(None,2), dtype='float32', name='Input')
#     out = LSTM(100, activation='relu', name='LSTM')(input)
# #     denseMean = Dense(2, name='DenseMean')(out)
# #     denseVariance = Dense(2, name='DenseVariance')(out)
    
#     #sampled_output = Lambda(sampling, name='sampling_layer')([denseMean, denseVariance])
    
#     #sampled_output = Lambda(sampling_alt, name='sampling_layer')([denseMean, denseVariance])
    
#     sampled_output = Dense(2, name='DummyDense')(out)
    
#     generator_pretraining = Model(input, sampled_output)
#     return generator_pretraining

In [167]:
trainer.pre_train_generator(g_epochs=35, g_pre_data_train=[sig_im_train_re, sig_im_train_re_t], g_pre_data_valid=[sig_im_valid_re, sig_im_valid_re_t], lr=1e-3)

Generator pre-training
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, None, 2)      0                                            
__________________________________________________________________________________________________
LSTM (LSTM)                     (None, 100)          41200       Input[0][0]                      
__________________________________________________________________________________________________
DenseMean (Dense)               (None, 2)            202         LSTM[0][0]                       
__________________________________________________________________________________________________
DenseVariance (Dense)           (None, 2)            202         LSTM[0][0]                       
______________________________________________________________________________________

In [131]:
# dummy_generator = GeneratorPretraining()

In [132]:
# g_adam = keras.optimizers.Adam(0.0001)
# dummy_generator.compile(g_adam, loss='mse')
# print('Generator pre-training')
# dummy_generator.summary()

In [133]:
# pre_train_g_filepath = 't_pre_train_g_weights'
# pre_train_g_c = [keras.callbacks.ModelCheckpoint(pre_train_g_filepath, monitor='val_loss',  save_best_only=True, save_weights_only=True)]
# dummy_generator.fit(
#     sig_im_train_re,
#     sig_im_train_re,
#     validation_data=(sig_im_valid_re, sig_im_valid_re),
#     callbacks=pre_train_g_c,
#     epochs=15)

In [168]:
sig_im_train_re[1].reshape(1,1,2)

array([[[ 1.143966 , -1.3628985]]], dtype=float32)

In [169]:
trainer.predict_pre_generator(sig_im_train_re[1].reshape(1,1,2))

array([[ 1.1439592, -1.3623166]], dtype=float32)

In [170]:
sig_im_test_re[0].reshape(1,1,2)

array([[[-1.1536084, -1.0670861]]], dtype=float32)

In [171]:
trainer.predict_pre_generator(sig_im_test_re[0].reshape(1,1,2))

array([[-1.1528805, -1.0671139]], dtype=float32)

In [172]:
print(trainer.predict_generator(sig_im_test_re[:1].reshape(1,1,2)))

[-1.15277487 -1.06715872]


In [150]:
print(trainer.predict_beta_generator(sig_im_test_re[:1].reshape(1,1,2)))

[-1.15267862 -1.06449991]


In [140]:
print(trainer.predict_curr_discriminator(rf_system))

[[0.46381766]]


G:\My Drive\Research\transmitter_impersonation_alt\WirelessSystem\utils.py:164: RuntimeWarning: overflow encountered in square
  y=x*(1+ a[0]*np.abs(x)**2 + a[1]*np.abs(x)**4 )
G:\My Drive\Research\transmitter_impersonation_alt\WirelessSystem\utils.py:164: RuntimeWarning: overflow encountered in power
  y=x*(1+ a[0]*np.abs(x)**2 + a[1]*np.abs(x)**4 )
G:\My Drive\Research\transmitter_impersonation_alt\WirelessSystem\utils.py:164: RuntimeWarning: overflow encountered in multiply
  y=x*(1+ a[0]*np.abs(x)**2 + a[1]*np.abs(x)**4 )
G:\My Drive\Research\transmitter_impersonation_alt\WirelessSystem\utils.py:157: RuntimeWarning: overflow encountered in square
  x=x/(np.mean(np.abs(x)**2))
G:\My Drive\Research\transmitter_impersonation_alt\WirelessSystem\utils.py:157: RuntimeWarning: invalid value encountered in true_divide
  x=x/(np.mean(np.abs(x)**2))
C:\Users\samur\Anaconda3\envs\tf_gpu\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the im

In [141]:
orig, pred=trainer.get_predicted_sequence(rf_system)

In [142]:
pred

array([[-5.19839135e-01,  5.78697555e-01],
       [-8.34919028e-01,  5.26195972e-01],
       [-6.86527192e-01,  5.00131734e-01],
       [ 2.31917500e-01,  6.50257270e-01],
       [ 5.17359420e-01,  5.24773290e-01],
       [ 1.58878824e-01, -1.54887082e-01],
       [-6.40204381e-01, -4.91981310e-01],
       [-2.17818489e-01, -6.85374108e-01],
       [ 6.19214372e-01, -4.80664587e-01],
       [ 9.07284121e-01, -5.67696659e-01],
       [ 4.35849234e-01, -4.76301908e-01],
       [-1.72903103e-01, -7.91097693e-02],
       [-5.64147902e-01,  4.98991367e-01],
       [-4.98627837e-01,  8.85126414e-01],
       [-4.37459485e-01,  5.17944016e-01],
       [-6.91570095e-01, -2.27929581e-01],
       [-6.68997865e-01, -5.60060902e-01],
       [ 2.22191909e-01,  5.19082044e-02],
       [ 6.27851511e-01,  5.76096457e-01],
       [ 1.00356752e-02,  2.21400840e-01],
       [-5.48581929e-01, -5.44385569e-01],
       [-3.41599466e-02, -7.99080254e-01],
       [ 6.99879904e-01, -4.63495842e-01],
       [ 5.

In [143]:
orig.shape

(256, 2)

In [144]:
pred.shape

(256, 2)

In [145]:
orig[252]

array([-0.5976945, -0.542979 ], dtype=float32)

In [146]:
pred[252]

array([-0.64518498, -0.61878346])

In [56]:
symb=rf_system.get_real_pretx_symbol_block(128)

In [58]:
symb.shape

(256, 2)

In [121]:
symb_r=symb.reshape(1,256,2)

In [127]:
st0=symb[0].reshape(1,1,2)
st1=symb[1].reshape(1,1,2)

In [123]:
symb[-1]

array([ 0.19163832, -0.47349837], dtype=float32)

In [124]:
symb_r[:,-1,:] = st0

In [126]:
st0

array([[[ 0.60103184, -0.5632952 ]]], dtype=float32)

In [125]:
symb_r[:,-1,:]

array([[ 0.60103184, -0.5632952 ]], dtype=float32)

In [129]:
sy_c=np.concatenate([symb_r, st1], axis=1)

In [131]:
st1

array([[[-0.14769381,  0.06279915]]], dtype=float32)

In [130]:
sy_c[:,-1,:]

array([[-0.14769381,  0.06279915]], dtype=float32)

In [173]:
trainer.train(rf_system)

1
0
1
[[[ 0.5167021 -0.5211162]]] [ 0.51649756 -0.52264973] [[0.5178832]]


ValueError: array must not contain infs or NaNs